# Redis tutorial

## Prerequisites

### Documentation

You will find all documentation for :
* [Redis commands](https://redis.io/commands)
* [Redis python client](https://redis-py.readthedocs.io/en/latest/)

Though the tutorial asks you to use Redis commands, they all have an equivalent in the Python client library.

### Import libraries

In [1]:
import redis

In [2]:
r = redis.Redis(host='redis', port=6379)

In [3]:
r.set('foo', 'bar')
value = r.get('foo')
print(value)

b'bar'


You can launch a terminal aside, connect to your server with a Redis client and check that the value is still present :

```bash
vagrant@nosql:~$ redis-cli
127.0.0.1:6379> get foo
"bar"
```

## I. Quick start

### Strings and numbers

**Q** : Store the string `hello world` in key `greetings` with a `SET` command. Then use a `GET` command to retrieve it.

In [4]:
r.set("greetings","hello world")
r.get("greetings")

b'hello world'

**Q** : Create a `connections` key with value 0. Find the `INCR` command to increment its value, then display it. 

Try to increment the value for `greetings`.

In [5]:
r.set("connections",0)
r.incr("connections") 
r.get("connections")

b'1'

### Lists

**Q** : Create a `colors` list with strings `red`, `black` and `blue`, using the `RPUSH` command.

In [6]:
r.rpush("colors","red")
r.rpush("colors","blue")
r.rpush("colors","black")

3

**Q** : Display the second element of the list with `LINDEX`

In [7]:
r.lindex("colors",1)

b'blue'

**Q** : Display the full list with `LRANGE`

In [8]:
r.lrange("colors",0,2)

[b'red', b'blue', b'black']

### Using sets

**Q** : Create a `nosql` set containing values `redis`, `mongodb` and `orientdb`, using the `SADD` command.

In [9]:
r.sadd("nosql","redis","mongodb","orientdb")

3

**Q** : Test if `mysql` is inside the set, using `SISMEMBER`

In [10]:
r.sismember("nosql","mysql")

0

**Q** : Add `hbase` to the set with `SADD`, then display all elements of the set with `SMEMBERS`. Try to add `redis` again and see what happens.

In [11]:
print(r.sadd("nosql","hbase"))
print(r.smembers("nosql"))
print(r.sadd("nosql","redis"))

1
{b'mongodb', b'redis', b'hbase', b'orientdb'}
0


### Using sorted sets

Sorted Sets are similar to Redis Sets with the unique feature of values stored in a set. The difference is, every member of a Sorted Set is associated with a score, that is used in order to take the sorted set ordered, from the smallest to the greatest score.

**Q** : Using `ZADD`, create a sorted set `top14` with the following entries :

```
score city
10 Agen 
33 Bordeaux 
32 Brive 
29 Castres 
38 Clermont 
24 Grenoble 
26 La Rochelle 
32 Montpellier 
14 Oyonnax 
20 Pau 
40 Racing 
22 Stade Français 
36 Toulon 
36 Toulouse
```

In [12]:
r.zadd("city", {"Agen":10, "Bordeaux":33, "Brive":32, "Castres":29, "Clermont":38, "Grenoble":24,"La Rochelle":26,'Montpellier':32,"Oyonnax": 14,"Pau": 20 ,"Racing":40, "Stade Français":22,"Toulon": 36,"Toulouse":36})

14

**Q** : Fetch the score for `Toulon` with `ZSCORE`, and its ranking with `ZRANK`.

In [13]:
print(r.zscore("city","Toulon"))
print(r.zrank("city","Toulon"))

36.0
10


**Q** : `ZRANK` starts at 0 and scores are sorted from lowest to highest, so we should use the `ZREVRANK` for a true ranking of our cities.

In [14]:
r.zrevrank("city","Toulon")

3

**Q** : Find the commands to display :
* the 3 best teams
* teams with more than 35 points

In [15]:
print(r.zrange("city",0,2,desc=True))# Best 3 teams 
print(r.zrangebyscore("city",min=35,max="+inf",withscores=True))

[b'Racing', b'Clermont', b'Toulouse']
[(b'Toulon', 36.0), (b'Toulouse', 36.0), (b'Clermont', 38.0), (b'Racing', 40.0)]


### Using dictionaries

**Q** : Create a dictionary `user:1` with `HMSET` with properties `id (1), name (Jean), age (22)`. Display it with `HGETALL`

In [16]:
r.hset("user:1",mapping={"id":1,"name":"Jean","age":"22"})
r.hgetall("user:1")

{b'id': b'1', b'name': b'Jean', b'age': b'22'}

**Q** : Add a `city (Lyon)` property and rename the user from `Jean` to `Paul`.

In [17]:
r.hset("user:1",mapping={"city":"Lyon","name":"Paul"})
r.hgetall("user:1")

{b'id': b'1', b'name': b'Paul', b'age': b'22', b'city': b'Lyon'}

## Modelling a query cache with Redis

You are modeling data from a REST query cache system with Redis.
A request is identified by the http method and its url (without the protocol).
The content of the request is stored as it is to be returned on demand.

**Q** : Insert a PUT request in the cache on http://my-api.fr/user/10 whose answer is {"id": 10, "name": "jean"}

In [18]:
r.set("PUT:my-api.fr/user/10", '{"id": 10, "name": "jean"}')

True

**Q** : Insert a GET request in the cache on http://my-api.fr/user?size=2 whose answer is[{"id": 10, "name": jean}, {"id": 11, "name": "Claire"}]

In [19]:
r.set("GET:my-api.fr/user?size=2", '[{"id": 10, "name": "jean"}, {"id": 11, "name": "Claire"}]')

True

**Q** : Create a set of cache request keys.

In [20]:
r.sadd("cache_registry", "PUT:my-api.fr/user/10", "GET:my-api.fr/user?size=2")

2

**Q** : Check if the GET request on http: //http://my-api.fr/user

In [21]:
exists = r.exists("GET:my-api.fr/user?size=2")
print(f"La requête est-elle en cache ? {'Oui' if exists else 'Non'}")

La requête est-elle en cache ? Oui


**Q** : Delete the PUT request on http://my-api.fr/user/10 from the cache.

In [22]:
r.delete("PUT:my-api.fr/user/10")
r.srem("cache_registry", "PUT:my-api.fr/user/10")

1

## Postquisites

The folloinwg command removes all data from your Redis cluster.

In [23]:
!redis-cli flushall

/bin/bash: line 1: redis-cli: command not found
